In [89]:
import pandas as pd
import numpy as np
import os

In [90]:
def fasta_reader(directory):
    seqs = []
    for file in os.listdir(directory):
        seq = ''
        filepath = directory + file
        print(file)
        f = open(filepath, 'r')
        if not file.startswith('.'):
            for i in f:
                if i[0] == '>':
                    if seq != '':
                        seqs.append(seq)
                        seq = ''
                else:
                    seq += i.strip()
            if seq != '':
                seqs.append(seq)
    return(seqs)

In [92]:
segments = fasta_reader('/scratch/interns202402/skluer/segments/')
nonseg = fasta_reader('/scratch/interns202402/skluer/NCnonsegmentedsingle/')

Wuhan_Fly_Virus_12.fna
Potato_black_ringspot_virus2.fna
Botryosphaeria_dothidea_chrysovirus_18.fna
Emaravirus_cercidis5.fna
Maize_rough_dwarf_virus15.fna
morning_glory_varicosavirus2.fna
Tobacco_yellow_crinkle_virus1.fna
Mammarenavirus_bearense2.fna
Dabieshan_tick_virus2.fna
Green_River_chinook_virus3.fna
Lake_Chad_virus4.fna
Wuhan_Insect_virus_23.fna
Jamestown_Canyon_virus2.fna
Wenzhou_tapeworm_virus_11.fna
Aspergillus_ochraceous_virus2.fna
Chum_salmon_reovirus_CS3.fna
Peruvian_horse_sickness_virus10.fna
Hubei_myriapoda_virus_92.fna
Bakau_virus1.fna
University_of_Giessen_virus1.fna
Saesbyeol_virus3.fna
Coccinia_mosaic_Tamil_Nadu_virus1.fna
Choristoneura_occidentalis_cypovirus_163.fna
Zegla_virus1.fna
Rice_ragged_stunt_virus9.fna
Minatitlan_virus1.fna
Emaravirus_tritici1.fna
Orthohantavirus_khabarovskense2.fna
Striped_jack_nervous_necrosis_virus1.fna
Operophtera_brumata_reovirus7.fna
Tenuivirus_persotritici2.fna
Cassia_yellow_blotch_virus2.fna
Warrego_virus5.fna
Orthobunyavirus_simbuen

In [93]:
len(segments)

4738

In [94]:
len(nonseg)

6569

In [95]:
encoding = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1], 'U':[0, 0, 0, 1], 'N':[0,0,0,0]
    , 'M':[0,0,0,0], 'R':[0,0,0,0], 'W':[0,0,0,0], 'S':[0,0,0,0], 'Y':[0,0,0,0], 'K':[0,0,0,0], 'V':[0,0,0,0], 'H':[0,0,0,0], 'D':[0,0,0,0],
    'B':[0,0,0,0], 'E':[0,0,0,0], 'F':[0,0,0,0], 'I':[0,0,0,0], 'J':[0,0,0,0], 'L':[0,0,0,0], 'O':[0,0,0,0], 
    'P':[0,0,0,0], 'Q':[0,0,0,0], 'X':[0,0,0,0], 'Z':[0,0,0,0]}

In [96]:
ls = ['P', 'help']
for i in ls:
    if encoding.get(i):
        print('yep')

yep


In [97]:
for i in segments:
    if len(i) >20000:
        segments.remove(i)

In [98]:
def one_hot_encode(sequence):
    encoding = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'T': [0, 0, 0, 1], 'U':[0, 0, 0, 1], 'N':[0,0,0,0]
    , 'M':[0,0,0,0], 'R':[0,0,0,0], 'W':[0,0,0,0], 'S':[0,0,0,0], 'Y':[0,0,0,0], 'K':[0,0,0,0], 'V':[0,0,0,0], 'H':[0,0,0,0], 'D':[0,0,0,0],
    'B':[0,0,0,0], 'E':[0,0,0,0], 'F':[0,0,0,0], 'I':[0,0,0,0], 'J':[0,0,0,0], 'L':[0,0,0,0], 'O':[0,0,0,0], 
    'P':[0,0,0,0], 'Q':[0,0,0,0], 'X':[0,0,0,0], 'Z':[0,0,0,0]}
    encoded_sequence = []

    for nucleotide in sequence:
        if encoding.get(nucleotide):
            encoded_sequence.append(encoding.get(nucleotide))
        else:
            encoded_sequence.append([0,0,0,0])

    feature_length = 20000

    while len(encoded_sequence) <20000:
        encoded_sequence.append([0,0,0,0])

    return encoded_sequence

In [99]:
one_hot_non = []
for i in nonseg:
    one_hot_non.append(one_hot_encode(i))

In [100]:
one_hot_seg = []
for i in segments:
    one_hot_seg.append(one_hot_encode(i))

In [103]:
y = [1] * len(one_hot_seg) + [0] * len(one_hot_non)

In [105]:
X = one_hot_seg + one_hot_non

In [107]:
from sklearn.model_selection import train_test_split

In [108]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [109]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [119]:
model = models.Sequential()
model.add(layers.Conv1D(20, 2, activation='relu', input_shape=(20000,4)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(20, 2, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(20, 2, activation='relu'))
model.add(layers.MaxPooling1D())
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation="sigmoid"))

In [120]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_27 (Conv1D)          (None, 19999, 20)         180       
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, 9999, 20)         0         
 g1D)                                                            
                                                                 
 conv1d_28 (Conv1D)          (None, 9998, 20)          820       
                                                                 
 max_pooling1d_19 (MaxPoolin  (None, 4999, 20)         0         
 g1D)                                                            
                                                                 
 conv1d_29 (Conv1D)          (None, 4998, 20)          820       
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 2499, 20)        

In [121]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [122]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
5/5 [==============================] - 2s 251ms/step - loss: 0.6695 - accuracy: 0.5188
Epoch 2/10
5/5 [==============================] - 1s 254ms/step - loss: 0.5725 - accuracy: 0.6687
Epoch 3/10
5/5 [==============================] - 1s 265ms/step - loss: 0.5543 - accuracy: 0.6750
Epoch 4/10
5/5 [==============================] - 1s 251ms/step - loss: 0.5246 - accuracy: 0.7000
Epoch 5/10
5/5 [==============================] - 1s 250ms/step - loss: 0.4841 - accuracy: 0.7750
Epoch 6/10
5/5 [==============================] - 1s 251ms/step - loss: 0.4738 - accuracy: 0.7250
Epoch 7/10
5/5 [==============================] - 1s 248ms/step - loss: 0.4203 - accuracy: 0.8687
Epoch 8/10
5/5 [==============================] - 1s 250ms/step - loss: 0.3918 - accuracy: 0.8562
Epoch 9/10
5/5 [==============================] - 1s 254ms/step - loss: 0.3699 - accuracy: 0.8750
Epoch 10/10
5/5 [==============================] - 1s 246ms/step - loss: 0.3282 - accuracy: 0.9187


In [124]:
test_loss, test_acc = model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 18ms/step - loss: 0.4790 - accuracy: 0.7500


In [115]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_24 (Conv1D)          (None, 19999, 20)         180       
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 9999, 20)         0         
 g1D)                                                            
                                                                 
 conv1d_25 (Conv1D)          (None, 9998, 20)          820       
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 4999, 20)         0         
 g1D)                                                            
                                                                 
 conv1d_26 (Conv1D)          (None, 4998, 20)          820       
                                                                 
 global_average_pooling1d_2   (None, 20)              

In [118]:
batch_sizes = [64, 128, 256]
learning_rates = [1e-3, 1e-4, 1e-5]
epochs = [5, 10, 15]

In [144]:
ls5 = []
for i in batch_sizes:
    for j in learning_rates:
        for k in epochs:

            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = j),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
            
            model.fit(X_train, y_train, batch_size = i, epochs=k)

            test_loss, test_acc = model.evaluate(X_test, y_test)

            ls5.append([i, j, test_loss, test_acc])

Epoch 1/5
3/3 [==============================] - 2s 399ms/step - loss: 0.3620 - accuracy: 0.8750
Epoch 2/5
3/3 [==============================] - 1s 395ms/step - loss: 0.3224 - accuracy: 0.8313
Epoch 3/5
3/3 [==============================] - 1s 395ms/step - loss: 0.2985 - accuracy: 0.9375
Epoch 4/5
3/3 [==============================] - 1s 395ms/step - loss: 0.2809 - accuracy: 0.9625
Epoch 5/5
2/2 [==============================] - 0s 19ms/step - loss: 0.4718 - accuracy: 0.7250
Epoch 1/10
3/3 [==============================] - 2s 406ms/step - loss: 0.3144 - accuracy: 0.8813
Epoch 2/10
3/3 [==============================] - 1s 404ms/step - loss: 0.2818 - accuracy: 0.8813
Epoch 3/10
3/3 [==============================] - 1s 402ms/step - loss: 0.2373 - accuracy: 0.9375
Epoch 4/10
3/3 [==============================] - 1s 402ms/step - loss: 0.2576 - accuracy: 0.9688
Epoch 5/10
3/3 [==============================] - 1s 402ms/step - loss: 0.2296 - accuracy: 0.9563
Epoch 6/10
3/3 [==========

KeyboardInterrupt: 

In [145]:
ls5

[[64, 0.001, 0.47184762358665466, 0.7250000238418579],
 [64, 0.001, 0.43870431184768677, 0.7749999761581421],
 [64, 0.001, 0.4028274416923523, 0.800000011920929],
 [64, 0.0001, 0.4029541015625, 0.800000011920929],
 [64, 0.0001, 0.39947694540023804, 0.824999988079071],
 [64, 0.0001, 0.3950316309928894, 0.8500000238418579],
 [64, 1e-05, 0.3925342261791229, 0.824999988079071],
 [64, 1e-05, 0.39212554693222046, 0.824999988079071],
 [64, 1e-05, 0.39355945587158203, 0.824999988079071],
 [128, 0.001, 0.6085519790649414, 0.75],
 [128, 0.001, 0.4092869758605957, 0.800000011920929],
 [128, 0.001, 0.38696563243865967, 0.824999988079071],
 [128, 0.0001, 0.40196818113327026, 0.800000011920929],
 [128, 0.0001, 0.4007951319217682, 0.875],
 [128, 0.0001, 0.4169038236141205, 0.875],
 [128, 1e-05, 0.41082820296287537, 0.875],
 [128, 1e-05, 0.40477389097213745, 0.8500000238418579]]

In [ ]:
with open('gridsearch_results.txt', 'w') as f:
    for line in lines:
        f.write(f"{line}\n")